In [89]:
%endpoint http://localhost:5820/MyDB/update
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>
PREFIX fhir_us_core: <http://hl7.org/fhir/us/core/StructureDefinition/>
PREFIX fhir_nhs: <https://fhir.hl7.org.uk/STU3/StructureDefinition/>

############################
### MODIFY FHIR ONTOLOGY ###
############################

## Modify properties so their range is their fhir:Reference.type instead of a fhir:Reference.
## (using Patient.generalPractitioner as an example)
INSERT DATA {
    fhir:Patient.generalPractitioner.range a           owl:Class ;
                                           owl:unionOf (fhir:Organization fhir:Practitioner fhir:PractitionerRole) .
};
# modify Patient.generalPractitioner so its range is the expected types
DELETE WHERE { fhir:Patient.generalPractitioner rdfs:range ?o };
INSERT DATA {
  fhir:Patient.generalPractitioner rdfs:range fhir:Patient.generalPractitioner.range .
};
# modify uses of Patient.generalPractitioner so they restrict the property using the expected types
DELETE { ?bnode owl:allValuesFrom ?o }
WHERE {
    ?bnode a                 owl:Restriction ;
           owl:onProperty    fhir:Patient.generalPractitioner ;
           owl:allValuesFrom ?o .
} ;
INSERT { ?bnode owl:allValuesFrom fhir:Patient.generalPractitioner.range }
WHERE {
    ?bnode a                 owl:Restriction ;
           owl:onProperty    fhir:Patient.generalPractitioner ;
} ;


#####################################
### ADD PROFILES TO FHIR ONTOLOGY ###
#####################################

# create USCorePatient (at least enough of it for these examples)
DELETE WHERE { fhir:USCorePatient ?p ?o };
DELETE WHERE { fhir_us_core:us-core-patient ?p ?o };
INSERT DATA {
    fhir_us_core:us-core-patient a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:maxCardinality 1
                        ] ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir_us_core:us-core-patient.us_core_birthsex ;
                          owl:maxCardinality 1
                        ] .
};

# create NHSPatient (at least enough of it for these examples)
DELETE WHERE { fhir:NHSPatient ?p ?o };
DELETE WHERE { fhir_nhs:CareConnect-Patient-1 ?p ?o };
INSERT DATA { fhir_nhs:CareConnect-Organization-1 a owl:Class; rdfs:subClassOf fhir:Organization . } ;
INSERT DATA { fhir_nhs:CareConnect-Practitioner-1 a owl:Class; rdfs:subClassOf fhir:Practitioner . } ;
INSERT DATA {
    fhir_nhs:CareConnect-Patient-1.generalPractitioner.range a          owl:Class ;
                                                             owl:unionOf (fhir_nhs:CareConnect-Organization-1 fhir_nhs:CareConnect-Practitioner-1) .
} ;
INSERT DATA {
    fhir_nhs:CareConnect-Patient-1.generalPractitioner
        a owl:ObjectProperty ;
        rdfs:subPropertyOf fhir:Patient.generalPractitioner ;
        rdfs:domain fhir_nhs:CareConnect-Patient-1 ;
        rdfs:range fhir_nhs:CareConnect-Patient-1.generalPractitioner.range
} ;
INSERT DATA {
    fhir_nhs:CareConnect-Patient-1 a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:cardinality 1
                        ] ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir_nhs:CareConnect-Patient-1.generalPractitioner ;
                          owl:allValuesFrom  fhir_nhs:CareConnect-Patient-1.generalPractitioner.range ] .
} ;

# create dummy Patient profile illustrating a specific example
DELETE WHERE { fhir:PatientProfileWithNameCardinalityOne ?p ?o };
INSERT DATA {
    fhir:PatientProfileWithNameCardinalityOne a owl:Class ;
        rdfs:subClassOf fhir:Patient ;
        rdfs:subClassOf [ a                  owl:Restriction ;
                          owl:onProperty     fhir:Patient.name ;
                          owl:cardinality 1
                        ] .
} ;

Endpoint set to: http://localhost:5820/MyDB/update
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

In [88]:
######################
# ATTRIBUTE PRESENCE #
######################

%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any
%display table

PREFIX fhir: <http://hl7.org/fhir/>

# Get all instances of profile pairs (cls1 cls2) where
# one profile has a property the other doesn't.
SELECT ?cls1 ?cls2 ?p1
WHERE {
    ?cls1 rdfs:subClassOf fhir:Patient .
    ?cls2 rdfs:subClassOf fhir:Patient .
    ?cls1 rdfs:subClassOf [ a      owl:Restriction ;
                                   owl:onProperty ?p1 ;
                          ] .
    FILTER NOT EXISTS {
        ?cls2 rdfs:subClassOf [ a      owl:Restriction ;
                                   owl:onProperty ?p1 ;
                              ] .
    }
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,p1
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient.us_core_birthsex
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient.us_core_birthsex
https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1.generalPractitioner
https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1.generalPractitioner


In [91]:
#########################
# ATTRIBUTE CARDINALITY #
#########################

%endpoint http://localhost:5820/MyDB/query
%auth basic spencer 12345
%format any 
%display table

PREFIX fhir: <http://hl7.org/fhir/>

# Get all instances of profile pairs (cls1 cls2) where the profiles
# restrict cardinality in a different way.
SELECT DISTINCT ?cls1 ?cls2 ?property ?card_restriction_1 ?card_1 ?card_restriction_2 ?card_2
WHERE {
    ?cls1 rdfs:subClassOf fhir:Patient .
    ?cls2 rdfs:subClassOf fhir:Patient .
    fhir:Patient rdfs:subClassOf [ a              owl:Restriction ;
                                   owl:onProperty ?property ;
                                 ] .
    ?cls1 rdfs:subClassOf  [ a                  owl:Restriction ;
                             owl:onProperty     ?property ;
                             ?card_restriction_1 ?card_1
                           ] .
    ?cls2 rdfs:subClassOf  [ a                  owl:Restriction ;
                             owl:onProperty     ?property ;
                             ?card_restriction_2 ?card_2
                           ] .
    
    FILTER (?cls1 != ?cls2) .
    FILTER (?card_restriction_1 IN (owl:cardinality, owl:maxCardinality, owl:minCardinality ) ) .
    FILTER (?card_restriction_2 IN (owl:cardinality, owl:maxCardinality, owl:minCardinality ) ) .
    FILTER (?card_restriction_1 != ?card_restriction_2 || ?card_1 != ?card_2) .
}

Endpoint set to: http://localhost:5820/MyDB/query
HTTP authentication: method=basic, user=spencer, passwd set
Request format: ANY
Display: table

cls1,cls2,property,card_restriction_1,card_1,card_restriction_2,card_2
https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#cardinality,1,http://www.w3.org/2002/07/owl#maxCardinality,1
http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#cardinality,1,http://www.w3.org/2002/07/owl#maxCardinality,1
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,https://fhir.hl7.org.uk/STU3/StructureDefinition/CareConnect-Patient-1,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#maxCardinality,1,http://www.w3.org/2002/07/owl#cardinality,1
http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient,http://hl7.org/fhir/PatientProfileWithNameCardinalityOne,http://hl7.org/fhir/Patient.name,http://www.w3.org/2002/07/owl#maxCardinality,1,http://www.w3.org/2002/07/owl#cardinality,1
